## QR Decomposition

In [1]:
import math
import numpy as np

from scipy import linalg

### Householder Method
Finding the QR decomposition relies on a method called the [householder method](https://www.quantstart.com/articles/QR-Decomposition-with-Python-and-NumPy/)

The algorithm work iteratively. At every stage you take the kth column from the bottom triangle of the matrix A. Then you apply the householder transform to the that vector against some identity matrix with the same dimensions

**Householder Transform**
$$H = I - 2\frac{vv^T}{v^Tv}$$

That spits out 

$$Q_k = \left(\begin{array}{cc} 
I_{k-1} & 0 \\
0 & Q'_{k}\\
\end{array}\right)$$

Then $R$ from its constituent parts
$$
R = Q_t \ldots Q_2Q_1A
$$

In [2]:
def qk_prime(a):
    '''
    rhs of householder transform computation for a vector a
    '''
    v = a / (a[0] + np.sign(a[0]) * np.linalg.norm(a))
    v[0] = 1
    tau = 2 / (v.T @ v)

    return tau * (v @ v.T)

def qr_decomposition(A):
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for j in range(0, n):
        # compute householder transform
        Qk = np.identity(m) 
        Qk[j:, j:] -= qk_prime(R[j:, j, np.newaxis])
        
        # update the orthogonal
        # and upper triangle matrices
        R = Qk @ R
        Q = Qk @ Q

    return Q[:n].T, np.triu(R[:n])

In [7]:
X = np.random.randn(3,3)
q, r = linalg.qr(X)
Q, R = qr_decomposition(X)
res = Q @ R
print(Q)

[[-0.24240934 -0.60049969  0.76199596]
 [-0.18092289 -0.74366333 -0.64360839]
 [-0.95315509  0.29387919 -0.0716268 ]]


### Applications

### Low Rank Approximation


In [81]:
def gen_matrix(r, c, rank):
    X = np.random.randn(r, c) * 10
    u, s, v = np.linalg.svd(X)
    return np.dot(u[:, :rank] * s[:rank], v[:, :rank].T)

In [92]:
ssr = lambda X, X_tilda : np.sum(np.power((X - X_tilda), 2))

# low rank approximations work great when
# the rank of the matrix is also low
rank_2 = gen_matrix(10, 10, 2)
Q, R = qr_decomposition(rank_2)
print("Sum of squared residuals (rank 2 approx vs rank 2 mat)", ssr(rank_2, Q[:, :2] @ R[:2, :]))

# when the rank of the matrix is higher, not as well
rank_10 = gen_matrix(10, 10, 10)
Q, R = qr_decomposition(rank_10)
print("Sum of squared residuals (rank 2 approx vs rank 10 mat)", ssr(rank_10, Q[:, :2] @ R[:2, :]))
print("Sum of squared residuals (rank 5 approx vs rank 10 mat)", ssr(rank_10, Q[:, :5] @ R[:5, :]))
print("Sum of squared residuals (rank 9 approx vs rank 10 mat)", ssr(rank_10, Q[:, :9] @ R[:9, :]))

Sum of squared residuals (rank 2 approx vs rank 2 mat) 7.884418228651131e-28
Sum of squared residuals (rank 2 approx vs rank 10 mat) 5505.414262381858
Sum of squared residuals (rank 5 approx vs rank 10 mat) 1831.2186819785015
Sum of squared residuals (rank 9 approx vs rank 10 mat) 5.830356852864459


### Refs
- [Toronto Course](https://www.cs.toronto.edu/~lczhang/csc338_20191/hw/a3/a3.html)
- [Scipy Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html)
- [Stack Overflow Post](https://stackoverflow.com/questions/53489237/how-can-you-implement-householder-based-qr-decomposition-in-python)